# Processing the acquired Raw Data into a consumable format

## Overview

### This notebook contains the step by step code and related documentation to process the raw page views and page count data acquired from the wikipedia REST API endpoints into a consumble format.

For the data processing process, the input is the raw wikipedia traffic numbers for the specified period. We could either start by running the englishwikipediatrafficanalysis_dataprocessing ipython notebook to get the raw files or could skip that process and grab the readily available offline raw data files in this repository. 

First we import all the required libraries/modules.

In [1]:
#json library has some good helper methods for working with JSON objects. 
#Since our raw data is in json format, we need the ability to deserialize json data into python objects for consumption.
import json
#Periodically we would need a way to check the intermediate results. We do that by printing the values of the variables.
#This is done using the display module from the IPython.core.display library.
from IPython.core.display import display
#pandas is another super useful python library that has many valueable data storage and manipulation functions. 
import pandas as pd

And then create and initialize some variables that can make the code easy to maintain, as well as set global level options to make the output more readable.

In [2]:
#Directory where the raw files exist
raw_data_dir = './data/raw/'
#Directory where the processed file will be saved to (at the end of this notebook if all steps are successful)
processed_data_dir = './data/processed/'

#Variables to hold the file names of the raw data files
pagecounts_desktop_site = 'pagecounts_desktop-site_200807-201607.json'
pagecounts_mobile_site = 'pagecounts_mobile-site_201410-201607.json'
pageviews_desktop = 'pageviews_desktop_201507-201709.json'
pageviews_mobile_app = 'pageviews_mobile-app_201507-201709.json'
pageviews_mobile_web = 'pageviews_mobile-web_201507-201709.json'

#Variables to hold the file names to contain the processed data at the end of successful execution of the steps in this notebook.
processedfilename = 'en-wikipedia_traffic_200801-201709.csv'

#preventing scientific notation of numbers in the results of executions in this notebook, for readability
pd.set_option('precision',2)

Following another good coding standard practice to make the code modular by creating helper functions for blocks of code that has a tendency to repeat two or more times. Again, this makes the code more maintainable by having only one place to change if there are any modifications needed. 

In this specific case, before we can start applying any processing or transformation, we need to read in the content from each of the five raw data files and into appropriate format. The following steps are repeated for each of these raw data files:

1. Open the file and read content line by line
2. For each line (the content is in a JSON object format), parse it into a dictionary with key and value format. The dictionary will have the following fields:
    * Project: the name of the project, en.wikipedia in this case for all data items
    * access-site: desktop, mobile-app or mobile-data for pageviews; and desktop-site or mobile-site for pagecounts
    * agent: (only relevant to pageviews) the value is user for all data items since we expect the data to be filtered during the acquisition process
    * granularity: monthly, set at acquisition step
    * timestamp: the timestamp for which the measurement is scoped to. The format is YYYYMMDD00 (NOTE: two trailing zeros)
    * count/views: measured value of traffic, field named count for pagecounts and views for pageviews
    
3. We only need the Year, Month and view count. The year and month are extracted from the timestamp field and the view count is the count or views field. So we keep only those two fields and discard the rest. We also parse the Year and Month from the timestamp field. 

4. At the end of this processing step, we have the Year, Month and the views count as a row which is each processed observation point.
    

In [3]:
#function definition for the file read and extraction steps
def extractviewsinformation(datareference):
    extracteddata = []
    
    #read the file line by line
    for line in open(raw_data_dir+datareference, 'r').readlines():
        #since a valid json requires the keys and values to be enclosed in " instead of ', we do the conversion
        line= line.replace("'","\"")
        #read the json formatted line into a dictionary format
        rawdataline=json.loads(line)

        #for each item in the dictionary, extract the necessary fields and discard the ones that are not needed
        for i in range(0,len(rawdataline)):
            datarow=rawdataline[i]
            #extract timestamp field
            timestamp = datarow['timestamp']
            #parse the year and month from the timestamp
            year = int(timestamp[0:4])
            month = int(timestamp[4:6])
            #extract the views or count (as appropriate for pageviews and pagecounts respectively)
            if datareference[4:9] == 'views':
                num_views = datarow['views']
            else:
                num_views = datarow['count']

            #make an observation with the year, month and the number of views
            listitem = [year,month,num_views]
            #add the observation to the list of observations
            extracteddata.append(listitem)
        
    #return the list of observations in a pandas dataframe format.
    return pd.DataFrame(extracteddata,dtype=object, columns=['year','month','num_views'])

Now that we have the function stub for the load and pre-processing extraction operation, we can apply this process for each of the five raw data files

In [4]:
#load number of views (page count) for desktop
pagecounts_desktop_site_data = extractviewsinformation(pagecounts_desktop_site)

#load number of views (page count) for mobile
pagecounts_mobile_site_data = extractviewsinformation(pagecounts_mobile_site)

#load number of views (page views) for desktop
pageviews_desktop_data = extractviewsinformation(pageviews_desktop)

#load number of views (page views) for mobile app
pageviews_mobile_app_data = extractviewsinformation(pageviews_mobile_app)

#load number of views (page views) for mobile web
pageviews_mobile_web_data = extractviewsinformation(pageviews_mobile_web)

In [5]:
#optionally display the intermediate results. 
# you can comment them out if you are running the code along and do not desire to see these intermediate results
display(pagecounts_desktop_site_data)
display(pagecounts_mobile_site_data)
display(pageviews_desktop_data)
display(pageviews_mobile_app_data)
display(pageviews_mobile_web_data)

,year,month,num_views
0,2008,7,5306302874
1,2008,8,5140155519
2,2008,9,5479533823
3,2008,10,5679440782
4,2008,11,5415832071
5,2008,12,5211708451
6,2009,1,5802681551
7,2009,2,5547320860
8,2009,3,6295159057
9,2009,4,5988817321


,year,month,num_views
0,2014,10,3091546685
1,2014,11,3027489668
2,2014,12,3278950021
3,2015,1,3485302091
4,2015,2,3091534479
5,2015,3,3330832588
6,2015,4,3222089917
7,2015,5,3334069483
8,2015,6,3038162463
9,2015,7,3254472695


,year,month,num_views
0,2015,7,4376666686
1,2015,8,4332482183
2,2015,9,4485491704
3,2015,10,4477532755
4,2015,11,4287720220
5,2015,12,4100012037
6,2016,1,4436179457
7,2016,2,4250997185
8,2016,3,4286590426
9,2016,4,4149383857


,year,month,num_views
0,2015,7,109624146
1,2015,8,109669149
2,2015,9,96221684
3,2015,10,94523777
4,2015,11,94353925
5,2015,12,99438956
6,2016,1,106432767
7,2016,2,92414130
8,2016,3,97899074
9,2016,4,81719003


,year,month,num_views
0,2015,7,3179131148
1,2015,8,3192663889
2,2015,9,3073981649
3,2015,10,3173975355
4,2015,11,3142247145
5,2015,12,3276836351
6,2016,1,3611404079
7,2016,2,3242448142
8,2016,3,3288785117
9,2016,4,3177044999


Note that none of the intermediate data lists that we have contain the observations for all the year-month combimations from July 2008 to September 2017. So we need a data structure that has a placeholder template for all these observations. The idea is to then fill in the observations from each of the intermediate views that we obtained in the previous step by doing a merge and filling with zeros for time periods when we do not have a measurement for the specific observation. We begin by creating this data structure and prefill the required year-month ranges. 

In [6]:
#list of years for which we need the observations
years = [2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]
#list of months numbers for each year for which we need the observations
months = [1,2,3,4,5,6,7,8,9,10,11,12]

#create the data structure that holds this year month combination
years_months = [[x,y] for x in years for y in months]

#the below line is only required if we want to limit the results to July 2008 - September 2017 window. 
#If you are following along running the code and desire to get the data starting January 2008, you can do the following:
#comment out the next line below and uncomment the following line
#this will set the appropriate year-month range
years_months = years_months[6:117]
#years_months = years_months[0:117]

#create a pandas dataframe for this data object
years_months = pd.DataFrame(years_months,dtype=object, columns=['year','month'])

#display this intermediate result.
display(years_months)

,year,month
0,2008,7
1,2008,8
2,2008,9
3,2008,10
4,2008,11
5,2008,12
6,2009,1
7,2009,2
8,2009,3
9,2009,4


Now we have the placeholder template with the required year month combinations filled in, and from the previous step we have all the observation points for the various access sites. We begin merging all of this into a single dataset format. The resultant dataset will contain the following fields:
1. Year: Year of the measurement point
2. Month: month of the measurement point
3. pagecount_all_views: pagecount for both desktop and mobile views
4. pagecount_desktop_views: pagecount for only desktop views
5. pagecount_mobile_views: pagecount for only mobile views
6. pageview_all_views: pageview for both desktop and mobile views
7. pageview_desktop_views: pageview for only desktop views
8. pageview_mobile_views: pageview for only mobile views

In [7]:
#fill in the number of views (pagecount) for desktop  
mergefirst = years_months.merge(pagecounts_desktop_site_data,how='left',on=['year','month'])

#fill in the number of views (pagecount) for mobile
mergesecond = pd.merge(mergefirst,pagecounts_mobile_site_data,how='left',on=['year','month'])

#fill in the number of views (pageview) for mobile web
mergethird = pd.merge(mergesecond,pageviews_mobile_web_data,how='left',on=['year','month'])

#fill in the number of views (pageview) for mobile app
mergefourth = pd.merge(mergethird,pageviews_mobile_app_data,how='left',on=['year','month'])

#fill in the number of views (pageview) for desktop
mergefifth = pd.merge(mergefourth,pageviews_desktop_data,how='left',on=['year','month'])

#for observation points when there was no data, replace NA with zeros.
mergefifth.fillna(0,inplace=True)

#Rename the columns as appropriate
mergefifth.columns = ['year','month','pagecount_desktop_views','pagecount_mobile_views','pageview_mobile_web_views', 'pageview_mobile_app_views', 'pageview_desktop_views']

#For pagecount, create a new field which contains the derived value of all views which includes both desktop and mobile 
mergefifth['pagecount_all_views'] = mergefifth['pagecount_desktop_views'] + mergefifth['pagecount_mobile_views']

#For pageview, create a new field which contains the derived value of all mobile views which combines both web and app 
mergefifth['pageview_mobile_views'] = mergefifth['pageview_mobile_web_views'] + mergefifth['pageview_mobile_app_views']

#For pageview, create a new field which contains the derived value of all views which includes both desktop and mobile 
mergefifth['pageview_all_views'] = mergefifth['pageview_mobile_views'] + mergefifth['pageview_desktop_views']

#we do not need all fields now, retain only the required ones. create the list of the fields that we need
columnstokeep = ['year','month','pagecount_all_views','pagecount_desktop_views','pagecount_mobile_views','pageview_all_views','pageview_desktop_views','pageview_mobile_views']

#create a new view with just the required columns from the list we created in the previous step
en_wikipedia_traffic_200801_201709=mergefifth[columnstokeep]

#visualize the final processed result before saving
display(en_wikipedia_traffic_200801_201709)

,year,month,pagecount_all_views,pagecount_desktop_views,pagecount_mobile_views,pageview_all_views,pageview_desktop_views,pageview_mobile_views
0,2008,7,5306302874,5306302874,0,0,0,0
1,2008,8,5140155519,5140155519,0,0,0,0
2,2008,9,5479533823,5479533823,0,0,0,0
3,2008,10,5679440782,5679440782,0,0,0,0
4,2008,11,5415832071,5415832071,0,0,0,0
5,2008,12,5211708451,5211708451,0,0,0,0
6,2009,1,5802681551,5802681551,0,0,0,0
7,2009,2,5547320860,5547320860,0,0,0,0
8,2009,3,6295159057,6295159057,0,0,0,0
9,2009,4,5988817321,5988817321,0,0,0,0


Now that we have the processed dataset, we are ready to now save it to a file. 

In [8]:
#save the processed result to a file and do not persist the row names
en_wikipedia_traffic_200801_201709.to_csv(processed_data_dir+processedfilename,index=False)